In [ ]:
import pandas as pd
import pathlib as pl
file = pl.Path("../recieved.csv")

headerNames = ["time", "recieverId", "tempInC", "pressure", "altitude", "humidity", "capacitance"]
recievedDf = pd.read_csv(file, names=headerNames)  # dF means DataFrame, a construct in pandas for tabled data
print(recievedDf)

In [ ]:
import plotly.express as px
# pd.options.plotting.backend = "plotly"
#  df[df['Time'] == 1]][['Wavelength', 'Absorption']]
px.scatter(recievedDf[recievedDf['recieverId']==1], x="time", y="altitude", title="Reciever 1")
px.scatter(recievedDf[recievedDf['recieverId']==3], x="time", y="altitude", title="Reviever 3")